In [30]:
# Enable auto-reload of modified modules
%reload_ext autoreload
%autoreload 2

from functions import *
import os


In [33]:
# Show all info for private key
def showPrivKeyInfo(priv_key):
    priv_key_byte = bytes.fromhex(priv_key)
    print('priv_key hex: ', priv_key)
    print('priv_key WIF uncompressed: ', getWIF(priv_key_byte))
    print('priv_key WIF compressed  : ', getWIF(priv_key_byte, compressed=True))

    publ_key_hex = getPubKey(priv_key_byte)
    print("Public key: ", publ_key_hex.hex())

    # get public key, uncompressed
    publ_key = getPubKeyFullUncompressed(publ_key_hex)
    print('Full public key uncompressed: ', publ_key.hex())
    print('Address uncompressed: ', getAddress(getPubKeyHashed(publ_key)))

    # get public key, compressed
    publ_key = getPubKeyFullCompressed(publ_key_hex)
    print('Full public key compressed  : ', publ_key.hex())
    print('Address compressed:   ', getAddress(getPubKeyHashed(publ_key)))

In [34]:
# priv_key = os.urandom(32).hex()
priv_key = '60cf347dbc59d31c1358c8e5cf5e45b822ab85b79cb32a9f3d98184779a9efc2'

showPrivKeyInfo(priv_key)

priv_key hex:  60cf347dbc59d31c1358c8e5cf5e45b822ab85b79cb32a9f3d98184779a9efc2
priv_key WIF uncompressed:  5JYvSurww2jTxmCeoN8T9QgRMWp45rre7WgFS76ae6Rgd1BnkC6
priv_key WIF compressed  :  KzTtuNKTTUeS186RqeFtQ7WzVYagcT46ojzEhoudUiwwsWtvokhD
Public key:  1e7bcc70c72770dbb72fea022e8a6d07f814d2ebe4de9ae3f7af75bf706902a7b73ff919898c836396a6b0c96812c3213b99372050853bd1678da0ead14487d7
Full public key uncompressed:  041e7bcc70c72770dbb72fea022e8a6d07f814d2ebe4de9ae3f7af75bf706902a7b73ff919898c836396a6b0c96812c3213b99372050853bd1678da0ead14487d7
Address uncompressed:  18CJdksq7eVGnENaackZDpMb4rgbc9YR7y
Full public key compressed  :  031e7bcc70c72770dbb72fea022e8a6d07f814d2ebe4de9ae3f7af75bf706902a7
Address compressed:    17JsmEygbbEUEpvt4PFtYaTeSqfb9ki1F1


In [35]:
validateAddress('18CJdksq7eVGnENaackZDpMb4rgbc9YR7y')

--------------------------------------
Bitcoin Address:  18CJdksq7eVGnENaackZDpMb4rgbc9YR7y
Base58 decoded:   004eec2d0c073f190c3ba671e2f9ce79574604fdcda22b6a2c
	Prefix:    00
	Hash:      4eec2d0c073f190c3ba671e2f9ce79574604fdcd
	Checksum:  a22b6a2c
--------------------------------------
Hash# 1  :  c31122682c264174e78440fc691827dd1f49d997b65252422913ddd2b9237558
Hash# 2  :  a22b6a2ca0ecc450ea2bed270f192a01170f216d17f982766c355f69efd10533
--------------------------------------
[TRUE]  checksum is valid!


In [21]:
# Parse addresses from http://addresses.loyce.club/

import csv

tsv_file = open("blockchair_bitcoin_addresses_and_balance_LATEST.tsv")
out_file = open("addresses_new.txt", 'a')
min_balance = 1000000 # in satoshis. 1 BTC = 100000000 satoshi.

read_tsv = csv.reader(tsv_file, delimiter="\t")
i = 0
c = 0
t = 0
for row in read_tsv:
    if i > 0 and row[0][0] == '1' and int(row[1]) >= min_balance:
        out_file.write(row[0]+'\n')
        c = c + 1
        t = t + int(row[1])
    i = i + 1
tsv_file.close()
out_file.close()
print("%s total funded addresses. %s P2PKH addresses with balance >= %s BTC (%s BTC)" % (i, c, min_balance/100000000, t/100000000))

40344866 total funded addresses. 4149271 P2PKH addresses with balance >= 0.01 BTC (10749349.14240564 BTC)


In [29]:
# Compare speed of different ECDSA libraries
import timeit

loops = 100

# https://github.com/tlsfuzzer/python-ecdsa
test_name = 'ecdsa'
setup = 'import ecdsa, binascii'
stmt  = 'secret = "60cf347dbc59d31c1358c8e5cf5e45b822ab85b79cb32a9f3d98184779a9efc2"; ecdsa.SigningKey.from_string(bytes.fromhex(secret), curve=ecdsa.SECP256k1).get_verifying_key().to_string().hex();'
print('[timeit] {0} loops, {1:.6} sec '.format( loops, timeit.timeit(stmt, setup, number=loops) ).ljust(50,' ') + test_name )

# https://github.com/starkbank/ecdsa-python
test_name = 'starkbank-ecdsa'
setup = 'from ellipticcurve.privateKey import PrivateKey'
stmt  = 'secret = "60cf347dbc59d31c1358c8e5cf5e45b822ab85b79cb32a9f3d98184779a9efc2"; PrivateKey.fromString(secret).publicKey().toString();'
print('[timeit] {0} loops, {1:.6} sec '.format( loops, timeit.timeit(stmt, setup, number=loops) ).ljust(50,' ') + test_name )

# https://github.com/ofek/coincurve
test_name = 'coincurve'
setup = 'from coincurve.keys import PrivateKey, PublicKey;'
stmt  = 'secret = "60cf347dbc59d31c1358c8e5cf5e45b822ab85b79cb32a9f3d98184779a9efc2"; PublicKey.from_valid_secret(bytes.fromhex(secret)).point();'
print('[timeit] {0} loops, {1:.6} sec '.format( loops, timeit.timeit(stmt, setup, number=loops) ).ljust(50,' ') + test_name )

[timeit] 100 loops, 0.120254 sec                  ecdsa
[timeit] 100 loops, 0.476535 sec                  starkbank-ecdsa
[timeit] 100 loops, 0.0078916 sec                 coincurve
